# init

In [ ]:
import numpy as np
import qcodes
import time
import sweep
import os


from qcodes.instrument_drivers.tektronix.Keithley_2400 import Keithley_2400
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes import VisaInstrument
from useful_functions import autorange_sr830s

In [ ]:
sweep.set_basedir('.')

## Keithleys

In [ ]:
k_bg1 = Keithley_2400('k_bg1', 'GPIB4::9::INSTR')
k_bg1.mode.set('VOLT')
#k_bg1.volt.set(0)
k_bg1.rangev.set(21)
k_bg1.rangei.set(1e-6)
k_bg1.compliancev.set(20)
k_bg1.compliancei.set(1e-6)
k_bg1.output.set(1)

k_bg1.volt.step = 0.005

k_tg2 = Keithley_2400('k_tg2', 'GPIB4::10::INSTR')
k_tg2.mode.set('VOLT')
#k_tg2.volt.set(0)
k_tg2.rangev.set(21)
k_tg2.rangei.set(1e-6)
k_tg2.compliancev.set(20)
k_tg2.compliancei.set(1e-6)
k_tg2.output.set(1)

k_tg2.volt.step = 0.005

k_bg2 = Keithley_2400('k_bg2', 'GPIB4::11::INSTR')
k_bg2.mode.set('VOLT')
#k_bg2.volt.set(0)
k_bg2.rangev.set(21)
k_bg2.rangei.set(1e-6)
k_bg2.compliancev.set(20)
k_bg2.compliancei.set(1e-6)
k_bg2.output.set(1)

k_bg2.volt.step = 0.005

k_tg3 = Keithley_2400('k_tg3', 'GPIB4::12::INSTR')
k_tg3.mode.set('VOLT')
#k_tg3.volt.set(0)
k_tg3.rangev.set(21)
k_tg3.rangei.set(1e-6)
k_tg3.compliancev.set(20)
k_tg3.compliancei.set(1e-6)
k_tg3.output.set(1)

k_tg3.volt.step = 0.005


heater = Keithley_2400('heater', 'GPIB4::12::INSTR')
heater.mode.set('VOLT')
#k_tg3.volt.set(0)
heater.rangev.set(21)
heater.rangei.set(1e-3)
heater.compliancev.set(20)
heater.compliancei.set(1e-6)
heater.output.set(1)

heater.volt.step = 0.005

## Lockins

In [ ]:
sr830_1 = SR830('sr830_1', 'GPIB1::1::INSTR')
sr830_2 = SR830('sr830_2', 'GPIB1::2::INSTR')
sr830_3 = SR830('sr830_3', 'GPIB1::3::INSTR')
sr830_4 = SR830('sr830_4', 'GPIB1::4::INSTR')

In [ ]:
sr830s = [(sr830_1, 100), 
          (sr830_2, 100), 
          (sr830_3, 100), 
          (sr830_4, -1E6),]


for i, (srs, gain) in enumerate(sr830s):
    print(i, srs.X(), gain)

## Magnet

In [ ]:
magnet = VisaInstrument('magnet', 'GPIB3::13::INSTR')
magnet.set_terminator('\r\n')
magnet.write("CONF:FIELD:UNITS 1")
magnet.write("CONF:COIL 0.116783")
magnet.write("CONF:RAMP:RATE:UNITS 0")
magnet.write("CONF:RAMP:RATE:SEG 1")
magnet.write("CONF:RAMP:RATE:FIELD 1,0.001,14")
magnet.write("CONF:PS 0")
magnet.write("CONF:QU:DET 1")
magnet.write("CONF:CURR:LIM 120")
magnet.write("CONF:VOLT:LIM 3.5")
magnet.write("CONF:STAB 59")
magnet.write("RAMP")

def set_b(mag, b):
    mag.write(f'CONF:FIELD:TARG {b}')
    time.sleep(1)
    mag.ask('STATE?') # Magnet somtimes retains things in its buffer
    time.sleep(0.01)
    while int(mag.ask('STATE?')) == 1:
        time.sleep(0.5)


def get_b(mag):
    ret = mag.ask('FIELD:MAG?') # Magnet somtimes retains things in its buffer
    time.sleep(0.01)
    ret = mag.ask('FIELD:MAG?')
    while len(ret) == 0:
        time.sleep(0.1)
        ret = mag.ask('FIELD:MAG?')
    return float(ret)

mag = qcodes.Parameter('mag', unit='T', get_cmd=lambda: get_b(magnet), set_cmd=lambda b: set_b(magnet, b))
print(f'Field: {mag()}')


## Thermometry

In [ ]:
def read_temperature():
    content = []
    retries = 5
    while len(content) == 0 and retries > 0:
        with open('/Users/dgglab/Desktop/control900/control/Release/temp.txt') as f:
            content = f.readlines()
        content = [x.strip() for x in content]
        if len(content) == 0:
            time.sleep(0.005)
            retries = retries - 1
    if retries == 0:
        print('Timed out waiting for temperature')
        content = ['nan']
    return float(content[0])
temperature = qcodes.Parameter('T', unit='K', get_cmd=read_temperature)
print(f'Temperature: {temperature()}')


def read_mag_temperature():
    content = []
    retries = 5
    while len(content) == 0 and retries > 0:
        with open('/Users/dgglab/Desktop/TempDataStorage/Fridge_Magnet.txt', 'rb') as f:
            try:  # catch OSError in case of a one line file 
                f.seek(-2, os.SEEK_END)
                while f.read(1) != b'\n':
                    f.seek(-2, os.SEEK_CUR)
                content = f.readline().decode().split('\t')
            except OSError:
                time.sleep(0.005)
                retries = retries - 1
    if retries == 0:
        print('Timed out waiting for temperature')
        content = ['nan']
    return float(content[-3])

mag_temperature = qcodes.Parameter('T', unit='K', get_cmd=read_mag_temperature)
print(f'Mag Temperature: {mag_temperature()}')


# Example station

In [ ]:
sr830s_2_autorange = [sr830_3, sr830_0, sr830_1, sr830_2]
s = sweep.Station()
s.fp(temperature)
s.fp(k_bg1.curr)
s.fp(k_tg2.curr)
for sr, gain in sr830s:
    s.fp(sr.X, gain)
    s.fp(sr.P)
s.register_run_before(autorange_sr830s, [sr830s_2_autorange])

#s.sweep(mag, np.linspace(0.5, 0.6, 26), delay=1)